In [16]:
import os
import glob
import time
import random
import string
import numpy as np
from numpy import array
from pickle import dump
from pickle import load
from tkinter import *
from tkinter import filedialog
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding, Dropout
import tensorflow as tf

Using TensorFlow backend.


In [17]:
#Configure GPUs for tensorflow
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [18]:
def load_doc(filename):
    # open the file
    file = open(filename, 'r', encoding='latin-1')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [19]:
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [20]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'a+')
    file.write(data)
    file.close()

In [21]:
# reduce length of incorrect lines to 11
def clean_sequences(sequences):
    count=0
    incorrect_lines = list()
    for i in sequences:
        if len(i)>11:
            while len(sequences[count])!=11:
                sequences[count].pop(-1)
        count+=1
    return sequences

In [22]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [25]:
def text2sequences(file):

    os.chdir(r'C:\Users\Aleksander\Desktop\4th term\advanced python\Text generation with LSTM\texts')
    # load document
    doc = load_doc(file)

    # clean document
    tokens = clean_doc(doc)
    print(tokens[:200])
    print('Total Tokens: %d' % len(tokens))
    print('Unique Tokens: %d' % len(set(tokens)))

    # organize into sequences of tokens
    length = 10 + 1
    sequences = list()
    for i in range(length, len(tokens)):
        # select sequence of tokens
        seq = tokens[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
    print('Total Sequences: %d' % len(sequences))
    # save sequences to file
    os.chdir(home_directory)
    save_doc(sequences, 'sequences.txt')

In [26]:
# remove "sequences.txt" if it exists
if os.path.exists('sequences.txt'):
    os.remove('sequences.txt')

home_directory = os.getcwd()
    
os.chdir(r'C:\Users\Aleksander\Desktop\4th term\advanced python\Text generation with LSTM\texts')
files = glob.glob('*.txt')

for file in files:
    text2sequences(file)
    
# load sequences
doc = load_doc('sequences.txt')
lines = doc.split('\n')

['call', 'me', 'ishmael', 'some', 'years', 'mind', 'how', 'long', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i', 'thought', 'i', 'would', 'sail', 'about', 'a', 'little', 'and', 'see', 'the', 'watery', 'part', 'of', 'the', 'world', 'it', 'is', 'a', 'way', 'i', 'have', 'of', 'driving', 'off', 'the', 'spleen', 'and', 'regulating', 'the', 'circulation', 'whenever', 'i', 'find', 'myself', 'growing', 'grim', 'about', 'the', 'mouth', 'whenever', 'it', 'is', 'a', 'damp', 'drizzly', 'november', 'in', 'my', 'soul', 'whenever', 'i', 'find', 'myself', 'involuntarily', 'pausing', 'before', 'coffin', 'warehouses', 'and', 'bringing', 'up', 'the', 'rear', 'of', 'every', 'funeral', 'i', 'meet', 'and', 'especially', 'whenever', 'my', 'hypos', 'get', 'such', 'an', 'upper', 'hand', 'of', 'me', 'that', 'it', 'requires', 'a', 'strong', 'moral', 'principle', 'to', 'prevent', 'me', 'from', 'deliberately', 'stepping', 'into', 'the

In [27]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [28]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

sequences = array(clean_sequences(sequences))
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [29]:
# define and train the model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)

model.save('model.h5')
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 50)            894100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 17882)             1806082   
Total params: 2,851,082
Trainable params: 2,851,082
Non-trainable params: 0
_________________________________________________________________


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
224634/224634 [==============================] - 92s 410us/step - loss: 6.9281 - accuracy: 0.0716s - loss: 6.9302 - 
Epoch 2/100
224634/224634 [==============================] - 107s 476us/step - loss: 6.4811 - accuracy: 0.0904
Epoch 3/100
224634/224634 [==============================] - 116s 517us/step - loss: 6.2542 - accuracy: 0.1039
Epoch 4/100
224634/224634 [==============================] - 156s 695us/step - loss: 6.0933 - accuracy: 0.1116
Epoch 5/100
224634/224634 [==============================] - 176s 782us/step - loss: 5.9516 - accuracy: 0.1175 - los
Epoch 6/100
224634/224634 [==============================] - 190s 844us/step - loss: 5.8160 - accuracy: 0.1220
Epoch 7/100
224634/224634 [==============================] - 178s 791us/step - loss: 5.6860 - accuracy: 0.1262
Epoch 8/100
224634/224634 [==============================] - 104s 464us/step - loss: 5.5643 - accuracy: 0.1305
Epoch 9/100
224634/224634 [==============================] - 92s 409us/step - loss: 5.44

224634/224634 [==============================] - 88s 391us/step - loss: 2.3387 - accuracy: 0.4847
Epoch 75/100
224634/224634 [==============================] - 85s 380us/step - loss: 2.3139 - accuracy: 0.4899
Epoch 76/100
224634/224634 [==============================] - 85s 378us/step - loss: 2.2925 - accuracy: 0.4945
Epoch 77/100
224634/224634 [==============================] - 86s 384us/step - loss: 2.2684 - accuracy: 0.4991
Epoch 78/100
224634/224634 [==============================] - 86s 382us/step - loss: 2.2481 - accuracy: 0.5017
Epoch 79/100
224634/224634 [==============================] - 82s 366us/step - loss: 2.2277 - accuracy: 0.5066
Epoch 80/100
224634/224634 [==============================] - 67s 300us/step - loss: 2.2043 - accuracy: 0.5115
Epoch 81/100
224634/224634 [==============================] - 293s 1ms/step - loss: 2.1795 - accuracy: 0.5151
Epoch 82/100
224634/224634 [==============================] - 150s 666us/step - loss: 2.1608 - accuracy: 0.5192
Epoch 83/100
2

In [30]:
# load cleaned text sequences
doc = load_doc('sequences.txt')
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

model = load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb'))

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [31]:
# get random "seed" sequence
seed_text = lines[random.randint(0,len(lines))]
print(seed_text + '\n')

seldom displays the back and more especially his flanks effaced in



In [32]:
# generate text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

great tubs and from the replacing of the blinds part of the harem dash into the wall where for inflexible in hook how then the sunset stars i light are reasonable to them their hearts with some subtle and glorious lay had been continued to leave out of consternations and
